In [1]:
# Imports
import cv2 as cv
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import glob
import pickle
import ml_metrics

In [2]:
# Parameters
NBINS = 64
COLORSPACE = cv.COLOR_BGR2Lab
DIVISIONS = 8

# Read the image database and store the features: MULTIRESOLUTION
database = []
for f in sorted(glob.glob('../database/*.jpg')):
    img = cv.imread(f, cv.IMREAD_COLOR)
    img = cv.cvtColor(img, COLORSPACE)
    # Level 0 histograms:
    hist_img = []
    npx = img.shape[0]*img.shape[1]
    hist_1 = cv.calcHist([img],[0],None,[NBINS],[0,256])/npx 
    hist_2 = cv.calcHist([img],[1],None,[NBINS],[0,256])/npx
    hist_3 = cv.calcHist([img],[2],None,[NBINS],[0,256])/npx
    hists = np.concatenate((hist_1,hist_2,hist_3))
    hist_img.append(hists)
    
    # Level 1 histograms
    for i in range(0,DIVISIONS):
        for j in range(0,DIVISIONS):
            # Compute the normalized histograms
            subimg = img[i*round(img.shape[0]/DIVISIONS):(i+1)*round(img.shape[0]/DIVISIONS)-1, 
                         j*round(img.shape[1]/DIVISIONS):(j+1)*round(img.shape[1]/DIVISIONS)-1]
            npx = subimg.shape[0]*subimg.shape[1]
            hist_1 = cv.calcHist([subimg],[0],None,[NBINS/2],[0,256])/npx 
            hist_2 = cv.calcHist([subimg],[1],None,[NBINS/2],[0,256])/npx
            hist_3 = cv.calcHist([subimg],[2],None,[NBINS/2],[0,256])/npx
            hists = np.concatenate((hist_1,hist_2,hist_3))
            hist_img.append(hists)
    
    flat_list = []
    for sublist in hist_img:
        for item in sublist:
            flat_list.append(item)
    database.append(flat_list)

print('Database has a length of ', len(database), 'images')

Database has a length of  279 images


In [3]:
# Read the query images
queries = []

for f in sorted(glob.glob('../qs/qsd1_w1/*.jpg')):
    img = cv.imread(f, cv.IMREAD_COLOR)
    img = cv.cvtColor(img, COLORSPACE)
    
    # Level 0 histograms:
    hist_img = []
    npx = img.shape[0]*img.shape[1]
    hist_1 = cv.calcHist([img],[0],None,[NBINS],[0,256])/npx 
    hist_2 = cv.calcHist([img],[1],None,[NBINS],[0,256])/npx
    hist_3 = cv.calcHist([img],[2],None,[NBINS],[0,256])/npx
    hists = np.concatenate((hist_1,hist_2,hist_3))
    hist_img.append(hists)
    
    # Multilevel histograms
    for i in range(0,DIVISIONS):
        for j in range(0,DIVISIONS):
            # Compute the normalized histograms
            subimg = img[i*round(img.shape[0]/DIVISIONS):(i+1)*round(img.shape[0]/DIVISIONS)-1, 
                         j*round(img.shape[1]/DIVISIONS):(j+1)*round(img.shape[1]/DIVISIONS)-1]
            npx = subimg.shape[0]*subimg.shape[1]
            hist_1 = cv.calcHist([subimg],[0],None,[NBINS/2],[0,256])/npx 
            hist_2 = cv.calcHist([subimg],[1],None,[NBINS/2],[0,256])/npx
            hist_3 = cv.calcHist([subimg],[2],None,[NBINS/2],[0,256])/npx
            hists = np.concatenate((hist_1,hist_2,hist_3))
            hist_img.append(hists)
    flat_list = []
    for sublist in hist_img:
        for item in sublist:
            flat_list.append(item)
    queries.append(flat_list)

print('Query set has ' + str(len(queries)) + ' images')


# Load the ground truth for the query set
gt = pickle.load(open('../qs/qsd1_w1/gt_corresps.pkl','rb'))

Query set has 30 images


In [4]:
final_ranking = np.zeros( (len(queries),10), dtype=float )
for i in range(0, len(queries)):
    ranking = np.zeros((10, 2), dtype=float)
    for j in range(0, len(database)):
        # Compute the distance metric
        dist = np.sum(np.sqrt(np.multiply(np.array(database[j]),np.array(queries[i]))))
        # Check the ranking and update it
        if (dist > min(ranking[:, 1])):
            # Add the distance and the id to the db
            idx = np.argmin(ranking[:, 1])
            ranking[idx, 0] = j
            ranking[idx, 1] = dist
    # Store the closest K images
    for j in range(0, 10):
        idx = np.argmax(ranking[:, 1])
        final_ranking[i, j] = ranking[idx, 0]
        ranking[idx, :] = [0, 0]


In [5]:
# Apply the MAP@K metric
ml_metrics.mapk(gt,final_ranking.tolist(),10)

0.8114285714285714

In [6]:
len(queries[0])

6336